# Webscraping at Zolo Real State Website.

### Getting the links to each building at Zolo website.

In [ ]:
"""
 Second way --- using the "webdriver-manager"
 we will be able to always download the latest version of the "chromedriver" 
 using our Google Chrome version as a reference.
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#allow us to navigate trough any page and to open a html archive in our pc.
#driver.get(r"https://www.zolo.ca/")
list_links=[]

for page in range(1,100):
    #define the link of the current page to be opened.
    current_page = 'https://www.zolo.ca/page-'+str(page)
    #open a page based on the link formed.
    driver.get(current_page)
    #print(current_page)
    #gets all the articles to a list.
    articles = driver.find_elements(By.TAG_NAME, 'article')
    #inside of each article it is looking for the "a hef" where we got the link to the detailed page.
    for article in articles:
        try:
            link = article.find_element(By.TAG_NAME, 'a').get_attribute('href')
            list_links.append(link)
            print(link)
        except:
            print('erro')
    #best order of elements -> ID > XPATH > CLASS_NAME
    #sleep(1)
    
    if list_links == 19999:
        break
        

#creating the dataframe to input into sheets.
df_links = pd.DataFrame(columns=['links_zolo'], data=list_links)



In [130]:
df_links.shape

(1862, 1)

### Sistematizando a montagem do dataframe

In [159]:
df_dados = pd.DataFrame(columns=['price'])
list_ = []

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

for link in df_links['links_zolo']:
    try:
        driver.get(link)
        price = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[1]').text
        address = driver.find_element(By.XPATH, '').text
        city = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[1]').text
        neighborhood = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[2]').text
        bed = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[1]').text
        bath = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[2]').text
        sale_or_rent = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[2]/div[1]').text
        size = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[3]/dd/span').text
        re_type = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[2]/div[1]').text
        except:
            re_type = 'NA'

        
df = pd.DataFrame(columns=['link', 'price', 'address', 'city', 'neighborhood', 'bed', 'bath', 're_type', 'size'])

for i in range(len(list_links)):
    current_link = link
    current_price = price
    current_address = address
    current_city = city
    current_neighborhood = neighborhood
    current_bed = bed
    current_bath = bath
    current_re_type = re_type
    current_size = size

df.loc[i] = [current_link, current_price, current_address, current_city, current_neighborhood, current_bed, current_bath, 
             current_re_type, current_size]
    


In [160]:
len(list_)

38

In [161]:
df = pd.DataFrame(data=list_, columns=['current_scrap'])

In [162]:
df['current_scrap'].value_counts().head(20)

For Rent    28
For Sale    10
Name: current_scrap, dtype: int64

In [95]:
import gspread
import pandas as pd

CODE = '1b4DllN1xFdujCzYnaMMWyC4tBg_gdmzT6G9GkqGT9Jo'

gc = gspread.service_account(filename='key.json')

sh = gc.open_by_key(CODE)

ws = sh.worksheet('Página1')

ws.update([df_links.columns.values.tolist()] + df_links.values.tolist())

{'spreadsheetId': '1b4DllN1xFdujCzYnaMMWyC4tBg_gdmzT6G9GkqGT9Jo',
 'updatedRange': "'Página1'!A1:A1863",
 'updatedRows': 1863,
 'updatedColumns': 1,
 'updatedCells': 1863}

In [ ]:
/html/body/section[2]/div/div/section[1]/section[2]/div[3]/div[1] #For Sale
/html/body/section[2]/div/div/section[1]/section[2]/div[2]/div[1] #For Rent
/html/body/section[2]/div/div/section[1]/section[2]/div[3]/div[1] #For Sale 

In [123]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#driver.get('https://www.zolo.ca/toronto-real-estate/120-varna-drive/612')

driver.get('https://www.zolo.ca/toronto-real-estate/50-thomas-riley-road/1106')


price = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[1]').text
city = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[1]').text
neighborhood = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[2]').text
re_type = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[2]/div[1]').text
bed = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[1]').text
bath = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[2]').text
type_ = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[1]/dd/span').text
style = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[2]/dd/span').text
size = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[3]/dd/span').text

print('price',price)
print('city',city)
print('neighborhood', neighborhood)
print('re_type', re_type)
print('bed', bed)
print('bath', bath)
print('type_', type_)
print('style', style)
print('size', size)

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="listing"]/div/div/section[1]/section[2]/div[3]/div[1]"}
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x000000010098bae8 chromedriver + 4958952
1   chromedriver                        0x00000001009837b3 chromedriver + 4925363
2   chromedriver                        0x00000001005451d6 chromedriver + 475606
3   chromedriver                        0x0000000100588a20 chromedriver + 752160
4   chromedriver                        0x0000000100588c81 chromedriver + 752769
5   chromedriver                        0x00000001005ccf64 chromedriver + 1032036
6   chromedriver                        0x00000001005aec3d chromedriver + 908349
7   chromedriver                        0x00000001005ca25b chromedriver + 1020507
8   chromedriver                        0x00000001005ae9e3 chromedriver + 907747
9   chromedriver                        0x00